In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import os

This is the code fot the simple model that uses Mnist

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split = ['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [3]:
#ds_train = train_ds_gen
#ds_test = train_ds_gen

In [4]:
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

In [5]:
ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

In [6]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data = ds_test
)

Epoch 1/6
469/469 [==============================] - 14s 12ms/step - loss: 0.3550 - sparse_categorical_accuracy: 0.9013 - val_loss: 0.1956 - val_sparse_categorical_accuracy: 0.9425
Epoch 2/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1650 - sparse_categorical_accuracy: 0.9533 - val_loss: 0.1424 - val_sparse_categorical_accuracy: 0.9596
Epoch 3/6
469/469 [==============================] - 2s 4ms/step - loss: 0.1218 - sparse_categorical_accuracy: 0.9652 - val_loss: 0.1134 - val_sparse_categorical_accuracy: 0.9666
Epoch 4/6
469/469 [==============================] - 2s 3ms/step - loss: 0.0957 - sparse_categorical_accuracy: 0.9722 - val_loss: 0.1009 - val_sparse_categorical_accuracy: 0.9701
Epoch 5/6
469/469 [==============================] - 2s 4ms/step - loss: 0.0776 - sparse_categorical_accuracy: 0.9768 - val_loss: 0.0881 - val_sparse_categorical_accuracy: 0.9730
Epoch 6/6
469/469 [==============================] - 2s 5ms/step - loss: 0.0644 - sparse_categorical_ac

From here there is my attempt at creating my datagenerator. First I save the files in a directory called 'data'.

In [8]:
#os.mkdir('data')

In [9]:
(mnist_train, mnist_test)= tfds.load(
    'mnist',
    split = ['train', 'test'],
    shuffle_files=True,
    as_supervised=True
)

In [10]:
iter_train = mnist_train.as_numpy_iterator()
iter_test = mnist_test.as_numpy_iterator()

In [11]:
lista_train = []
lista_test = []

In [12]:
#I use np.save, but there probably are some problems with the format of files
#as I get a Warning of Visible Deprecation. I tried solving it by allowing pickle

offset = 0
for i, el in enumerate(iter_train):
  np.save('data/arr'+str(i), el, allow_pickle=True)
  lista_train.append('arr'+str(i))
  offset = i + 1
for i, el in enumerate(iter_test):
  np.save('data/arr'+str(i + offset), el, allow_pickle=True)
  lista_test.append('arr'+str(i + offset))


/usr/local/lib/python3.9/dist-packages/numpy/lib/npyio.py:518: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


In [13]:
#just checking that the lists are correct
print(lista_train[-1])
print(lista_test[0])

arr59999
arr60000


In [14]:
labels_train = {e[1] : e[0] for e in enumerate(lista_train)}
labels_test = {e[1] : e[0] for e in enumerate(lista_test)}

In [15]:
partition = {'train' : lista_train, 'test' : lista_test}

In [16]:
#This is the code for the custom datagenerator, that should go in a separate script

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), n_channels=1,
                 n_classes=10, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('data/' + ID + '.npy', allow_pickle = True)

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [17]:
params = {'dim': (28, 28),
          'batch_size': 64,
          'n_classes': 6,
          'n_channels': 1,
          'shuffle': True}

In [18]:
train_ds_gen = DataGenerator(partition['train'], labels_train, **params)

In [19]:
test_ds_gen = DataGenerator(partition['test'], labels_test, **params)

In [20]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit_generator(
    generator = train_ds_gen,
    epochs=6,
    validation_data = test_ds_gen
)

<ipython-input-20-83ef88b6e7d6>:12: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


ValueError: ignored